In [1]:
import cv2
import os
import torch
import glob
from helpers import *
from vot.region.io import read_trajectory
from SegTracker import SegTracker
from model_args import segtracker_args, sam_args, aot_args

DATASET = '../vot-mixformer-sam/sequences'
# SEQ = 'cat'
SEQ = 'ants1'
seq = os.path.join(DATASET, SEQ)

In [2]:
imgs_paths = sorted(glob.glob(os.path.join(seq, 'color/*.jpg')))
gt_paths =  sorted(glob.glob(os.path.join(seq, 'groundtruth_obj*.txt')))

first_img = cv2.imread(imgs_paths[0])
initial_mask = np.zeros(first_img.shape[:2], dtype=np.uint8)

for i, gt_path in enumerate(gt_paths, 1):
    curr_obj = read_trajectory(gt_path)[0]
    bounds = curr_obj.bounds()
    initial_mask[bounds[1]:bounds[3]+1, bounds[0]:bounds[2]+1] = curr_obj.mask * i

In [3]:
seg_tracker = SegTracker(segtracker_args, sam_args, aot_args)
seg_tracker.restart_tracker()

/usr/mvl2/itdfh/anaconda3/envs/vot/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from ./ckpt/groundingdino_swint_ogc.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
SegTracker has been initialized


In [4]:
io_args = {
  'output_video': f'output_videos/{SEQ}.mp4',
  'output_masked_frame_dir': 'masked_frames',
}

In [5]:
# create dir to save predicted mask and masked frame
output_mask_dir = 'output'
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(io_args['output_masked_frame_dir'], exist_ok=True)

pred_list = []
masked_pred_list = []

torch.cuda.empty_cache()
sam_gap = seg_tracker.sam_gap
frame_idx = 0

with torch.cuda.amp.autocast():
    for img_path in imgs_paths:
        frame_name = os.path.basename(img_path).split('.')[0]
        frame = cv2.imread(img_path)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

        if frame_idx == 0:
            seg_tracker.add_reference(frame, initial_mask)
            torch.cuda.empty_cache()
            pred_mask = initial_mask
            
        # elif (frame_idx % sam_gap) == 0:
        #     seg_mask = seg_tracker.seg(frame)
        #     torch.cuda.empty_cache()
            
        #     track_mask = seg_tracker.track(frame)
        #     # find new objects, and update tracker with new objects
        #     new_obj_mask = seg_tracker.find_new_objs(track_mask,seg_mask)
        #     save_prediction(new_obj_mask, output_mask_dir, f'{frame_name}_new.png')
        #     pred_mask = track_mask + new_obj_mask
        #     # Seg_Tracker.restart_tracker()
        #     seg_tracker.add_reference(frame, pred_mask)
        else:
            pred_mask = seg_tracker.track(frame, update_memory=True)
        torch.cuda.empty_cache()
        
        
        save_prediction(pred_mask, output_mask_dir, f'{frame_name}.png')
        pred_list.append(pred_mask)

        print("processed frame {}, obj_num {}".format(frame_idx, seg_tracker.get_obj_num()),end='\r')
        frame_idx += 1
    print('\nfinished')

processed frame 324, obj_num 6
finished


In [6]:
##################
# Visualization
##################

# draw pred mask on frame and save as a video
height, width = pred_list[0].shape
fourcc =  cv2.VideoWriter_fourcc(*"mp4v")

fps=30
out = cv2.VideoWriter(io_args['output_video'], fourcc, fps, (width, height))

frame_idx = 0
for img_path in imgs_paths:
    frame_name = os.path.basename(img_path).split('.')[0]
    frame = cv2.imread(img_path)
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    pred_mask = pred_list[frame_idx]
    masked_frame = draw_mask(frame, pred_mask)
    masked_pred_list.append(masked_frame)
    cv2.imwrite(f"{io_args['output_masked_frame_dir']}/{frame_name}.png", masked_frame[:, :, ::-1])

    masked_frame = cv2.cvtColor(masked_frame,cv2.COLOR_RGB2BGR)
    out.write(masked_frame)
    print('frame {} writed'.format(frame_name),end='\r')
    frame_idx += 1
out.release()
print("\n{} saved".format(io_args['output_video']))
print('\nfinished')

torch.cuda.empty_cache()

frame 00000325 writed
out.mp4 saved

finished
